In [4]:
import os
import pandas as pd
import xml.etree.ElementTree as ET

# Diretório contendo os arquivos XML
diretorio_xmls = r'C:\macena\NFCe'

# Dicionário para armazenar os dados por tag
dados_por_tag = {}

# Percorre todos os arquivos no diretório
for arquivo in os.listdir(diretorio_xmls):
    if arquivo.endswith('.xml'):
        # Caminho completo do arquivo XML
        caminho_xml = os.path.join(diretorio_xmls, arquivo)

        # Lê o arquivo XML
        tree = ET.parse(caminho_xml)
        root = tree.getroot()

        # Percorre todas as tags do XML
        for tag in root.iter():
            # Ignora as tags de atributos
            if not tag.attrib:
                # Obtém o nome da tag
                nome_tag = tag.tag
                
                # Verifica se a tag já existe no dicionário
                if nome_tag in dados_por_tag:
                    # Adiciona os dados da tag no dicionário
                    dados_por_tag[nome_tag].append(tag.text)
                else:
                    # Cria uma nova entrada no dicionário para a tag
                    dados_por_tag[nome_tag] = [tag.text]

# Encontra o comprimento máximo dos arrays de dados
comprimento_maximo = max(len(dados) for dados in dados_por_tag.values())

# Preenche os arrays de dados com NaN para ter o mesmo comprimento
for dados in dados_por_tag.values():
    while len(dados) < comprimento_maximo:
        dados.append(None)

# Cria um DataFrame a partir do dicionário de dados
df = pd.DataFrame(dados_por_tag)

# Define o caminho do arquivo Excel de saída
caminho_excel = 'dados_xmls.xlsx'

# Salva o DataFrame como uma planilha Excel
df.to_excel(caminho_excel, index=False)

print(f"Arquivo Excel gerado com sucesso: {caminho_excel}")


Arquivo Excel gerado com sucesso: dados_xmls.xlsx
